# Intent
Ensure a class only has one instance, and provide a global point of access to it.
# Motivation
It's important for some classes to have exactly one instance. Although there can be
many printers in a system, there should be only one printer spooler. There should be
only one file system and one window manager. A digital filter will have one A/D
converter. An accounting system will be dedicated to serving one company.
How do we ensure that a class has only one instance and that the instance is easily
accessible? A global variable makes an object accessible, but it doesn't keep you from
instantiating multiple objects.
A better solution is to make the class itself responsible for keeping track of its sole
instance. The class can ensure that no other instance can be created (by intercepting
requests to create new objects), and it can provide a way to access the instance. This is
the Singleton pattern.
# Use the Singleton pattern when
• there must be exactly one instance of a class, and it must be accessible to clients
from a well-known access point. \
• when the sole instance should be extensible by subclassing, and clients should
be able to use an extended instance without modifying their code. 

In [8]:
class SingletonMeta(type):
    """
    The Singleton class can be implemented in different ways in Python. Some
    possible methods include: base class, decorator, metaclass. We will use the
    metaclass because it is best suited for this purpose.
    """

    _instances = {}

    def __call__(cls, *args, **kwargs):
        """
        Possible changes to the value of the `__init__` argument do not affect
        the returned instance.
        """
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]


class Singleton(metaclass=SingletonMeta):
    def some_business_logic(self):
        """
        Finally, any singleton should define some business logic, which can be
        executed on its instance.
        """
        pass

In [9]:
s1 = Singleton()
s2 = Singleton()

id(s1) == id(s2)

True

## Method 1: A decorator

In [ ]:
def singleton(class_):
    instances = {}
    def getinstance(*args, **kwargs):
        if class_ not in instances:
            instances[class_] = class_(*args, **kwargs)
        return instances[class_]
    return getinstance

@singleton
class MyClass(BaseClass):
    pass

Pros

* Decorators are additive in a way that is often more intuitive than multiple inheritance.

Cons

* While objects created using MyClass() would be true singleton objects, MyClass itself is a function, not a class, so you cannot call class methods from it. \
Also for
x = MyClass(); \
y = MyClass(); \
t = type(n)(); \
then x == y but x != t && y != t

## Method 2: A base class

In [ ]:
class Singleton(object):
    _instance = None
    def __new__(class_, *args, **kwargs):
        if not isinstance(class_._instance, class_):
            class_._instance = object.__new__(class_, *args, **kwargs)
        return class_._instance

class MyClass(Singleton, BaseClass):
    pass

Pros

* It's a true class

Cons

* Multiple inheritance - eugh! __new__ could be overwritten during inheritance from a second base class? One has to think more than is necessary.

## Method 3: A metaclass

In [ ]:
class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

#Python2
class MyClass(BaseClass):
    __metaclass__ = Singleton

#Python3
class MyClass(BaseClass, metaclass=Singleton):
    pass

Pros

* It's a true class
* Auto-magically covers inheritance
* Uses __metaclass__ for its proper purpose (and made me aware of it)

Cons

* Are there any?

## Method 4: decorator returning a class with the same name

In [ ]:
def singleton(class_):
    class class_w(class_):
        _instance = None
        def __new__(class_, *args, **kwargs):
            if class_w._instance is None:
                class_w._instance = super(class_w,
                                    class_).__new__(class_,
                                                    *args,
                                                    **kwargs)
                class_w._instance._sealed = False
            return class_w._instance
        def __init__(self, *args, **kwargs):
            if self._sealed:
                return
            super(class_w, self).__init__(*args, **kwargs)
            self._sealed = True
    class_w.__name__ = class_.__name__
    return class_w

@singleton
class MyClass(BaseClass):
    pass

Pros

* It's a true class
* Auto-magically covers inheritance

Cons

* Is there not an overhead for creating each new class? Here we are creating two classes for each class we wish to make a singleton. While this is fine in my case, I worry that this might not scale. Of course there is a matter of debate as to whether it aught to be too easy to scale this pattern...
* What is the point of the _sealed attribute
* Can't call methods of the same name on base classes using super() because they will recurse. This means you can't customize __new__ and can't subclass a class that needs you to call up to __init__.